# Libraries

In [1]:
# global libraries
import os
import sys
import random
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display
# Libraries
from io import StringIO, BytesIO
import requests
from pytz import timezone
from PIL import Image

In [2]:
# CNN Libraries
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
tf.__version__

'2.5.0'

In [4]:
# Tensorflow libraries
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D, MaxPooling2D, BatchNormalization, Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam

# Siamese Libraries
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Concatenate, Dot, Embedding

# Preprocess

In [5]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# change directory to dataset path
%cd gdrive/My Drive/visually aware/visually-aware-recommender-system/notebooks

/content/gdrive/My Drive/visually aware/visually-aware-recommender-system/notebooks


# Load Data

In [7]:
# Load data
dataset = np.load("AmazonMenWithImgPartitioned.npy", 
                  encoding="bytes", allow_pickle=True)
[user_train, user_validation, user_test, items, user_num, item_num] = dataset

# Hyper parameters

In [8]:
# Network params
# image size
image_width = 224
image_height = 224

# latent dimensionality K
latent_dimensionality = 50

# weight decay - conv layer
lambda_cnn = 0.001
# weight decay - fc layer
lambda_fc = 0.001
# regularizer for theta_u
lambda_u = 1.0

In [9]:
# Training params
# epoch params
learning_rate = 1e-4
validation_sample_count = 100

# ImageNet Transfer Learning Model

In [10]:
# Transfer learning - use pretraint InceptionV3 model
def build_inceptionV3(width, height, depth, latent_dim, w_init="RandomNormal", fc_w_regularizer=None, b_init="RandomNormal"):
  input_shape = (height, width, depth)
  model = Sequential()
  inception = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
  inception.trainable = False
  model.add(inception)
  model.add(Flatten())
  model.add(Dense(latent_dim, kernel_initializer=w_init, kernel_regularizer=fc_w_regularizer, bias_initializer=b_init))
  return model

# Subsets

In [11]:
# get user information
user_num_original = user_num
user_train_original = user_train

In [12]:
# for each batch, force the number of users to be the same
batch_size = 128
batch_count = int(np.ceil(user_num_original / batch_size))

# one complete model will be linked to each user_subset
user_subsets = dict(zip(range(batch_count), np.array_split(range(user_num_original), batch_count)))

# Utils

In [13]:
# translate image from byte code to an array and normalize its values from 0-255 to -1 - 1
def image_translate(image_bytes,
                    image_width=224,
                    image_height=224):
    img = (np.uint8(np.asarray(Image.open(BytesIO(image_bytes)).convert("RGB").resize((image_height, image_width)))) - 127.5)  / 127.5
    return img

In [14]:
# get train batches which was sampled uniformly for each user
def uniform_train_sample_batch(user_train_ratings, item_images, image_width=224, 
                               image_height=224, sample=True, batch_size=None, user_idx=None):

    if batch_size is not None:
        users = range(batch_size)
    else:
        users = user_idx

    triplet_train_batch = {}

    # iterate over each user in subset
    for b in users:

        # training set
        if sample:
            u = random.randrange(len(user_train_ratings))
        else:
            u = b

        i = user_train_ratings[u][random.randrange(len(user_train_ratings[u]))][b'productid'] # get random obesrved item
        j = random.randrange(len(item_images)) # sample random item

        # while the sampled items observed by the user - sample other item until found non-observed item
        while j in [item[b'productid'] for item in user_train_ratings[u]]:
            j = random.randrange(len(item_images))

        image_i = image_translate(item_images[i][b'imgs'], 
                                  image_width, 
                                  image_height)
        image_j = image_translate(item_images[j][b'imgs'],
                                  image_width, 
                                  image_height)
        triplet_train_batch[u] = [image_i,
                                  image_j]
        
    return triplet_train_batch

In [15]:
# get validation batches which was sampled uniformly for each user
def uniform_validation_sample_batch(user_train_ratings,
                                    user_validation_ratings,
                                    item_images,
                                    image_width=224,
                                    image_height=224,
                                    validation_sample_count=1000,
                                    sample=True,
                                    batch_size=None,
                                    user_idx=None):
    """
    validation_sample_count (int): Number of not-observed items to sample to get the validation set for each user.
    """

    if batch_size is not None:
        users = range(batch_size)
    else:
        users = user_idx

    triplet_validation_batch = {}
    
    # iterate over each user in subset
    for b in users:

        if sample:
            u = random.randrange(len(user_train_ratings))
        else:
            u = b

        # validation set
        i = user_validation_ratings[u][0][b'productid'] # get the only observed item
         # get its image
        image_i = image_translate(item_images[i][b'imgs'], 
                                  image_width, 
                                  image_height)

        # create a set of all user observed items
        reviewed_items = set()
        for item in user_train_ratings[u]:
            reviewed_items.add(item[b'productid'])
        reviewed_items.add(user_validation_ratings[u][0][b'productid'])

        triplet_validation_batch[u] = []
        
        # search for a non-observed item
        got = False
        for j in random.sample(range(len(item_images)), validation_sample_count):
            if j not in reviewed_items:
                image_j = image_translate(item_images[j][b'imgs'],
                                          image_width, 
                                          image_height)
                triplet_validation_batch[u] = [image_i, image_j]
                got = True
            
            if got:
              break

    return triplet_validation_batch

In [16]:
# Define the loss function as ln(sigmoid) according to the BPR method
# BPR wants to maximize the loss function while Keras engine minimizes it so we added a subtraction 
def bpr_loss(label_matrix, prediction_matrix):
    return 1 - tf.reduce_sum(tf.math.log(tf.sigmoid(prediction_matrix)))

In [17]:
# Count the ratio of prediction value > 0
# i.e., predicting positive item score > negative item score for a user
def auc(label_tensor, prediction_tensor):
    return K.mean(K.switch(prediction_tensor > K.zeros_like(prediction_tensor),
                           K.ones_like(prediction_tensor),    # 1
                           K.zeros_like(prediction_tensor)))  # 0

# Siamese Network

In [19]:
# build the Siamese network model
def ConvSiameseNet(users_dim, width, height, depth,latent_dim, w_init="RandomNormal", cnn_w_regularizer=None, fc_w_regularizer=None, u_w_regularizer=None, b_init="RandomNormal"):

        # user ID input
        user_input = Input((1,), name='User')

        image_shape = (width, height, depth)

        # observed image
        left_input = Input(image_shape,
                           name="observed_image")
        # non-observed image
        right_input = Input(image_shape,
                            name="non_observed_image")

        # Build convnet to use in each siamese 'leg'
        conv_net = build_inceptionV3(width, height, depth, latent_dim, w_init, fc_w_regularizer, b_init)

        # preference layer
        preference_relationship = Dot(axes=1, name="score_rank")

        # Apply the pipeline to the inputs
        # call the convnet Sequential model on each of the input tensors
        # so params will be shared
        encoded_l = conv_net(left_input)
        encoded_r = conv_net(right_input)

        # merge the two encoded inputs through the L1 distance
        L1_dist = tf.keras.layers.Subtract()([encoded_l, encoded_r])
        
        # retrieve the single user preference
        user_vec = Flatten(name='FlattenUsers')(Embedding(user_num+1, latent_dimensionality, name = 'User-Embedding', embeddings_initializer= tf.keras.initializers.RandomUniform(0,0.01),
                                                          input_length=1, embeddings_regularizer=u_w_regularizer)(user_input))
        
        # get the preference score
        prediction = preference_relationship([user_vec, L1_dist])
        
        # Create the model
        model = Model(inputs=[user_input, left_input, right_input],
                      outputs=prediction)
        
        model.summary()
        
        
        return model

# Build Model

In [20]:
# build the model with the regularization hyper-parameters and initializers
conv_siamese_net = ConvSiameseNet(users_dim=len(user_subsets[0]),
                                        width=image_width,
                                        height=image_height,
                                        depth=3,
                                        latent_dim=latent_dimensionality,
                                        cnn_w_regularizer=l2(lambda_cnn),
                                        fc_w_regularizer=l2(lambda_fc),
                                        u_w_regularizer=l2(lambda_u),
                                        w_init = tf.keras.initializers.GlorotNormal())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
User (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
observed_image (InputLayer)     [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
non_observed_image (InputLayer) [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 50)        1712250     User[0][0]                       
______________________________________________________________________________________________

In [21]:
# compile the model
optimizer = Adam(learning_rate)
conv_siamese_net.compile(loss=bpr_loss,
                         optimizer=optimizer,
                         metrics=[auc])

# Train

In [22]:
# get train subset using the uniform sampling
def get_train(user_subset, user_train):
  train = uniform_train_sample_batch(
      sample=False,
      user_idx=user_subset,
      user_train_ratings=user_train,
      item_images=items,
      image_width=image_width,
      image_height=image_height)
  return train

In [23]:
# get validation subset using the uniform sampling
def get_validation(user_subset, user_train):
  validation = uniform_validation_sample_batch(
      sample=False,
      user_idx=user_subset,
      user_train_ratings=user_train,
      user_validation_ratings=user_validation,
      item_images=items,
      image_width=image_width,
      image_height=image_height,
      validation_sample_count=validation_sample_count)
  return validation

In [24]:
# create triplets lists with the user ID and the two images - train
def get_train_triplet_lists(train):
    user_placeholder = []
    observed_image = []
    not_observed_image = []
    
    for u, triplet in train.items():
        user_placeholder.append(u)
        observed_image.append(triplet[0])
        not_observed_image.append(triplet[1])
        
    # label set does not exist in BPR, so we give Keras with a dummy label set
    labels = np.ones((len(train), 1), dtype=int)
    
    return user_placeholder, observed_image, not_observed_image, labels

In [25]:
# create triplets lists with the user ID and the two images - validation
def get_val_triplet_lists(validation):
    user_placeholder = []
    observed_image = []
    not_observed_image = []

    for u, triplet in validation.items():      
      user_placeholder.append(u)
      observed_image.append(triplet[0])
      not_observed_image.append(triplet[1])
      
    # label set does not exist in BPR, so we give Keras with a dummy label set
    labels = np.ones((len(validation), 1), dtype=int)

    return user_placeholder, observed_image, not_observed_image, labels

In [26]:
# generate train batches throughout the training phase - yields triplet batches
def generator():
  batches = range(batch_count)
  num_batches_in_curr_epoch = 0
  user_subset_idx = 0
  while True:
    user_subset = user_subsets[user_subset_idx] # get batch users
    user_train = {k: v for k, v in user_train_original.items() if k in user_subset}
    train = get_train(user_subset, user_train)
    user_placeholder, observed_image, not_observed_image, labels = get_train_triplet_lists(train)
    
    yield [np.asarray(user_placeholder), np.asarray(observed_image), np.asarray(not_observed_image)], np.asarray(labels)
    
    num_batches_in_curr_epoch+=1
    user_subset_idx+=1
    
    if num_batches_in_curr_epoch >= batch_count:
      # reset the lists
      num_batches_in_curr_epoch = 0
      user_subset_idx = 0

In [27]:
# generate validation batches throughout the evaluation phase - yields triplet batches
def val_generator():
  batches = range(batch_count)
  num_batches_in_curr_epoch = 0
  user_subset_idx = 0
  while True:
    user_subset = user_subsets[user_subset_idx] # get batch users
    validation = get_validation(user_subset, user_validation)
    user_placeholder, observed_image, not_observed_image, labels = get_val_triplet_lists(validation)
    
    yield [np.asarray(user_placeholder), np.asarray(observed_image), np.asarray(not_observed_image)], np.asarray(labels)

    num_batches_in_curr_epoch+=1
    user_subset_idx+=1
    
    if num_batches_in_curr_epoch >= batch_count:
      # reset the lists
      num_batches_in_curr_epoch = 0
      user_subset_idx = 0

In [28]:
# data generators for training and avaluation
train_generator = generator()
validation_generator = val_generator()

In [30]:
# conv_siamese_net.fit(x=train_generator, steps_per_epoch = num_batches, validation_data=validation_generator, validation_steps=batch_count, epochs = 20)

In [31]:
# Training 
val_scores = []
history = []
model_path = "conv_siamese_net_trained_model_ImageNet_20_epochs"

for i in range(2):
  history.append(conv_siamese_net.fit(x=train_generator, steps_per_epoch = batch_count, epochs = 10)) # train
  val_scores.append(conv_siamese_net.evaluate(validation_generator, steps = batch_count)) # validation
  print(f"validation score: {val_scores}") 
  conv_siamese_net.save(model_path, save_traces=True, include_optimizer=True) # save model

Epoch 1/10
268/268 [==============================] - 422s 1s/step - loss: 98.1863 - auc: 0.5735
Epoch 2/10
268/268 [==============================] - 375s 1s/step - loss: 88.6041 - auc: 0.6182
Epoch 3/10
268/268 [==============================] - 373s 1s/step - loss: 83.5907 - auc: 0.6771
Epoch 4/10
268/268 [==============================] - 374s 1s/step - loss: 78.8341 - auc: 0.7270
Epoch 5/10
268/268 [==============================] - 377s 1s/step - loss: 73.7081 - auc: 0.7702
Epoch 6/10
268/268 [==============================] - 374s 1s/step - loss: 69.8830 - auc: 0.7995
Epoch 7/10
268/268 [==============================] - 377s 1s/step - loss: 66.6871 - auc: 0.8210
Epoch 8/10
268/268 [==============================] - 381s 1s/step - loss: 63.4660 - auc: 0.8399
Epoch 9/10
268/268 [==============================] - 377s 1s/step - loss: 61.0695 - auc: 0.8516
Epoch 10/10
268/268 [==============================] - 356s 1s/step - loss: 97.8938 - auc: 0.6985
validation score: [[97.893783

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: conv_siamese_net_trained_model_ImageNet_30_epochs/assets


INFO:tensorflow:Assets written to: conv_siamese_net_trained_model_ImageNet_30_epochs/assets


Epoch 1/10
268/268 [==============================] - 382s 1s/step - loss: 57.7197 - auc: 0.8697
Epoch 2/10
268/268 [==============================] - 382s 1s/step - loss: 55.6358 - auc: 0.8815
Epoch 3/10
268/268 [==============================] - 388s 1s/step - loss: 54.4811 - auc: 0.8865
Epoch 4/10
268/268 [==============================] - 381s 1s/step - loss: 53.3653 - auc: 0.8932
Epoch 5/10
268/268 [==============================] - 379s 1s/step - loss: 52.3402 - auc: 0.8958
Epoch 6/10
268/268 [==============================] - 377s 1s/step - loss: 51.7496 - auc: 0.8997
Epoch 7/10
268/268 [==============================] - 377s 1s/step - loss: 50.6808 - auc: 0.9030
Epoch 8/10
268/268 [==============================] - 378s 1s/step - loss: 49.4612 - auc: 0.9081
Epoch 9/10
268/268 [==============================] - 378s 1s/step - loss: 48.8940 - auc: 0.9093
Epoch 10/10
268/268 [==============================] - 348s 1s/step - loss: 102.4114 - auc: 0.7091
validation score: [[97.89378

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: conv_siamese_net_trained_model_ImageNet_30_epochs/assets


INFO:tensorflow:Assets written to: conv_siamese_net_trained_model_ImageNet_30_epochs/assets


In [32]:
# remove garbage
import gc
gc.collect()

724471

# Load Model

In [ ]:
# load pretrained model
model_path = "conv_siamese_net_trained_model_ImageNet_20_epochs"
conv_siamese_net = tf.keras.models.load_model(model_path, custom_objects={'auc': auc, 'bpr_loss': bpr_loss})

In [ ]:
# compile the model
optimizer = Adam(learning_rate)
conv_siamese_net.compile(loss=bpr_loss,
                         optimizer=optimizer,
                         metrics=[auc])

# Test

In [33]:
# test set generator - yields batches through evaluation
def test_generator():
  batches = range(batch_count)
  num_batches_in_curr_epoch = 0
  user_subset_idx = 0
  while True:
    user_subset = user_subsets[user_subset_idx] # get batch users
    test = get_validation(user_subset, user_test)  
    user_placeholder, observed_image, not_observed_image, labels = get_val_triplet_lists(test)
    
    yield [np.asarray(user_placeholder), np.asarray(observed_image), np.asarray(not_observed_image)], np.asarray(labels)

    num_batches_in_curr_epoch+=1
    user_subset_idx+=1
    
    if num_batches_in_curr_epoch >= batch_count:
      # reset the lists
      num_batches_in_curr_epoch = 0
      user_subset_idx = 0

In [34]:
# evaluate on test set
test_gen = test_generator()
conv_siamese_net.evaluate(test_gen, steps = batch_count)

268/268 [==============================] - 355s 1s/step - loss: 103.1939 - auc: 0.7071


[103.19387817382812, 0.7071347832679749]